In [1]:
import random
import os
import threading
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, wait

from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np
import plotly.express as px

In [2]:
import pandas as pd
from pandas.core.common import flatten

In [3]:
layout_path = './../data/layout 18x25_6.json'
MAX_WORKERS = 24
USE_CATEGORY_FRO_CHECKS = False

In [4]:
selected_categories = [
 'bakery',
 'beverages',
 'breakfast',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'meat seafood',
 'pantry',
 'produce',
 'snacks']

# Preprocessing

In [5]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...
2019496,3390742,199430,16,3,18,5.0,83,8,0,4,produce,fresh vegetables
2019497,458285,128787,42,2,19,3.0,115,1,1,7,beverages,water seltzer sparkling water
2019498,458285,128787,42,2,19,3.0,32,2,1,4,produce,packaged produce
2019499,458285,128787,42,2,19,3.0,32,3,1,4,produce,packaged produce


In [6]:
df = df[['order_id', 'user_id', 'order_number', 'product_id', 'product_name', 'department', 'department_id']]
df

,order_id,user_id,order_number,product_id,product_name,department,department_id
0,2425083,49125,1,17,baking ingredients,pantry,13
1,2425083,49125,1,91,soy lactosefree,dairy eggs,16
2,2425083,49125,1,36,butter,dairy eggs,16
3,2425083,49125,1,83,fresh vegetables,produce,4
4,2425083,49125,1,83,fresh vegetables,produce,4
...,...,...,...,...,...,...,...
2019496,3390742,199430,16,83,fresh vegetables,produce,4
2019497,458285,128787,42,115,water seltzer sparkling water,beverages,7
2019498,458285,128787,42,32,packaged produce,produce,4
2019499,458285,128787,42,32,packaged produce,produce,4


In [7]:
df['product_name'].unique()

array(['baking ingredients', 'soy lactosefree', 'butter',
       'fresh vegetables', 'yogurt', 'canned meals beans',
       'poultry counter', 'ice cream ice', 'fresh fruits', 'milk',
       'packaged cheese', 'bread', 'tea', 'bakery desserts',
       'frozen breakfast', 'cereal', 'eggs', 'buns rolls', 'cream',
       'water seltzer sparkling water', 'pickled goods olives',
       'packaged poultry', 'other creams cheeses',
       'honeys syrups nectars', 'coffee', 'refrigerated',
       'energy granola bars', 'soft drinks', 'latino foods',
       'plates bowls cups flatware', 'paper goods', 'oral hygiene',
       'diapers wipes', 'food storage', 'nuts seeds dried fruit', 'soap',
       'packaged vegetables fruits', 'hot dogs bacon sausage',
       'lunch meat', 'chips pretzels', 'meat counter',
       'fresh dips tapenades', 'prepared soups salads', 'condiments',
       'juice nectars', 'canned fruit applesauce',
       'preserved dips spreads', 'packaged produce',
       'canned jarr

In [8]:
# return tuple {orderId, list(items)} for single check
def get_order_items(order_id):
    order = df[df['order_id'] == order_id]
    if not USE_CATEGORY_FRO_CHECKS:
        return order_id, order['product_name'].unique().tolist()
    else:
        is_in_category = order['department'].apply(lambda x: x in selected_categories)
        return order_id, order[is_in_category]['product_name'].unique().tolist()

In [9]:
if USE_CATEGORY_FRO_CHECKS:
    df = df[df['department'].isin(selected_categories)]

In [10]:
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:10000]):
    check = get_order_items(check_id)
    check_list.append(check)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
len(check_list)

10000

In [12]:
if USE_CATEGORY_FRO_CHECKS:
    items_list = df[df['department'].apply(lambda x: x in selected_categories)]['product_name'].unique().tolist()
else:
    items_list = df['product_name'].unique().tolist()

In [13]:
len(items_list)

134

# Layout

In [14]:
layout_test = Layout(layout_path).get_empty_rack_layout()
layout_test

In [15]:
def random_layout():
    layout = Layout(layout_path)
    layout.set_item_list(items_list)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [16]:
def layout_with_one_item(max_rack_level=4):
    layout = Layout(layout_path)
    layout.set_item_list(items_list)
    layout.set_max_rack_level(max_rack_level)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(layout.get_item_list()[0], (row, col), level=lev)
    return layout

In [17]:
layouts = []
for i in range(10):
    layouts.append(random_layout())

In [18]:
# dir = Path(os.getcwd()).parent
# layouts[2].display_in_window(home_dir=str(dir))

In [19]:
layouts[3]

# Evaluate

In [20]:
from helpers.estimation_helpers import evaluate_layout, thread_func, calculate_score

In [21]:
for layout in layouts:
    print(evaluate_layout(layout, check_list[0][1], use_item_count=True))

(88, False)
(61, False)
(84, False)
(62, False)
(69, False)
(65, False)
(65, False)
(79, False)
(84, False)
(72, False)


In [22]:
#dirr = Path(os.getcwd()).parent
#layouts[2].display_in_window(home_dir=str(dirr))

# Mutate layout

In [23]:
def select_n_best_layouts(layouts, checks, n_best=2, start_pos=None, reward_type='max', debug=False, use_item_count=False, weights=(500, 150, 350)):
    res = dict()
    f_res = dict()
    
    # shuffle checks
    #random.shuffle(checks)
    
    def format_debug_string(score):
        return f'Path: {score["path"]}, Invalid: {score["invalid"]}, Uniformity: {score["rack_uniformity"]},{score["tile_uniformity"]}'
    
    def format_short_debug_string(score):
        return f'P:{score["path"]}, I:{score["invalid"]}, U:{score["rack_uniformity"]},{score["tile_uniformity"]}'
    
    def optimal_score(score):
        return score['invalid'] * 10000 + score['rack_uniformity'] * 100 + score['tile_uniformity']
    
    if MAX_WORKERS > 1:
        futures = list()
        with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
            for layout in layouts:
                futures.append(executor.submit(thread_func, layout, checks, start_pos, use_item_count))
        
        #done, not_done = wait(futures, return_when='ALL_COMPLETED')
        for future in futures:
            # store modified layout as its path count is changed
            score, n_layout = future.result()
            res[n_layout] = score
    else:
        for layout in layouts:
            score, n_layout = thread_func(layout, checks, start_pos, use_item_count)
            res[n_layout] = score
    f_res = dict()
    if reward_type == 'max':
        res = sorted(res.items(), key=lambda x: x[1]['path'], reverse=True)
    elif reward_type == 'min':
        for key in res.keys():
            res[key]['path'] = res[key]['path'] + res[key]['invalid']*1000
        res = sorted(res.items(), key=lambda x: x[1]['path'])
    elif reward_type == 'valid':
        res = sorted(res.items(), key=lambda x: x[1]['invalid'])
    elif reward_type == 'uniformity':
        res = sorted(res.items(), key=lambda x: optimal_score(x[1]))
    elif reward_type == 'score':
        for key in res.keys():
            f_res[key] = calculate_score(res[key], key, checks, weights)
        f_res = sorted(f_res.items(), key=lambda x: x[1], reverse=False)

    if debug:
        if reward_type == 'score':
            print(f"Scores: ", end='')
            print([f[1] for f in f_res[:n_best]])
            return [x[0] for x in f_res[:n_best]], [x[1] for x in f_res[:n_best]], [res[x[0]]['missing_items'] for x in f_res[:n_best]]
        print([format_short_debug_string(x[1]) for x in res[:n_best]])
    return [x[0] for x in res[:n_best]], [x[1] for x in res[:n_best]], [x[1]['missing_items'] for x in res[:n_best]]

In [24]:
#MAX_WORKERS = 1
new_layouts, hi, _ = select_n_best_layouts(layouts, check_list[:1], debug=True, reward_type='uniformity', use_item_count=True)

['P:61, I:0, U:980,565', 'P:75, I:0, U:981,562']


In [25]:
new_layouts

In [26]:
def mutate_layout(original_layout, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        new_layout.set_item_to_rack(random.choice(new_layout.get_item_list()), (row, col), level=lev)
    return new_layout

In [27]:
def mutate_uniformity(original_layout, alpha=0.01):
    def get_neighbours(layout, i, j, level):
        floor_pos = None
        left_item = None
        right_item = None
        for pos in [(i, j-1), (i, j+1), (i-1, j), (i+1, j)]:
            if 0 <= pos[0] < layout.shape[0] and 0 <= pos[1] < layout.shape[1]:
                if layout[pos[0]][pos[1]].type.name == 'FLOOR':
                    floor_pos = pos
                    break
        diff = (floor_pos[0] - i, floor_pos[1] - j)
        pos_left = (i - diff[1], j - diff[0])
        pos_right = (i + diff[1], j + diff[0])
        if layout[pos_left[0]][pos_left[1]].type.name == 'RACK':
            left_item = layout[pos_left[0]][pos_left[1]].items[level][0]
        if layout[pos_right[0]][pos_right[1]].type.name == 'RACK':
            right_item = layout[pos_right[0]][pos_right[1]].items[level][0]
        return left_item, right_item
    
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        left_item, right_item = get_neighbours(new_layout, row, col, lev)
                        # if both neighbours are not None
                        if left_item and right_item:
                            # if both neighbours are the same - set the same item to the current rack
                            if left_item == right_item:
                                new_layout.set_item_to_rack(left_item, (row, col), level=lev)
                            # if neighbours are different - set randomly one of them
                            else:
                                new_layout.set_item_to_rack(random.choice([left_item, right_item]), (row, col), level=lev)
                        # if one of the neighbours is None - set existing item to the current rack
                        elif left_item:
                            new_layout.set_item_to_rack(left_item, (row, col), level=lev)
                        elif right_item:
                            new_layout.set_item_to_rack(right_item, (row, col), level=lev)
    return new_layout

In [28]:
def mutate_missing_items(original_layout, missing_items_array, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    
    def get_present_and_missing_items(layout):
        present_items = dict()
        missing_items = []
        for row in range(layout.shape[0]):
            for col in range(layout.shape[1]):
                if layout[row][col].type.name == 'RACK':
                    for lev in range(layout.get_max_rack_level()):
                        if layout[row][col].items[lev][0] in present_items.keys():
                            present_items[layout[row][col].items[lev][0]] += 1
                        else:
                            present_items[layout[row][col].items[lev][0]] = 1
        for item in layout.get_item_list():
            if item not in present_items.keys():
                missing_items.append(item)
        return present_items, missing_items
    
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    present_items, missing_items_array = get_present_and_missing_items(new_layout)
                    most_popular = sorted(present_items.items(), key=lambda x: x[1], reverse=True)[:15]
                    most_popular = [x[0] for x in most_popular]
                    if len(most_popular) == 15 and len(missing_items_array) > 0:
                        # increased chance of mutation for popular items
                        if random.random() < alpha*2 and new_layout[row][col].items[lev][0] in most_popular:
                            item = random.choice(missing_items_array)
                            new_layout.set_item_to_rack(item, (row, col), level=lev)
                    else:
                        if random.random() < alpha:
                            item = random.choice(new_layout.get_item_list())
                            new_layout.set_item_to_rack(item, (row, col), level=lev)
                        
    return new_layout

In [29]:
def mutate_uniformity_for_rack(original_layout, alpha=0.01):
    new_layout = original_layout.copy()
    new_layout.reset_path_count()
    for row in range(new_layout.shape[0]):
        for col in range(new_layout.shape[1]):
            if new_layout[row][col].type.name == 'RACK':
                for lev in range(new_layout.get_max_rack_level()):
                    if random.random() < alpha:
                        items = [x[0] for x in new_layout[row][col].items]
                        if len(items) > 1:
                            items.pop(lev)
                        new_layout.set_item_to_rack(random.choice(items), (row, col), level=lev)
    return new_layout

In [30]:
temp = mutate_layout(new_layouts[0], alpha=0.1)

In [31]:
dir = Path(os.getcwd()).parent
temp.display_in_window(home_dir=str(dir))

# Genetic algorithm

In [32]:
layouts_array = []
for i in range(50):
    layouts_array.append(layout_with_one_item())

In [33]:
# evaluate, select, mutate, repeat
def genetic_algorithm(
        layouts_arr, 
        check_arr, 
        n_iter=100, 
        n_best=10, 
        alpha=0.01, 
        start_pos=None, 
        debug=False, 
        reward_type='max', 
        use_item_count=False, 
        weights=(500, 150, 350),
        rule_based_mutations=False,
        alpha_change_func: callable = None):
    history = []
    l_size = len(layouts_arr)
    check_arr.sort(key=lambda x: len(x[1]))
    mutation_type = ""
    for i in trange(n_iter):
        if debug:
            print(f'Iteration {i} \'{mutation_type}\'', end=' ')
        if alpha_change_func:
            alpha = alpha_change_func(i, n_iter, alpha)
        n_best_arr, hi, res_dict = select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count, weights=weights)
        history.append(hi)
        layouts_arr = []
        for l in n_best_arr:
            layouts_arr.append(l.copy().reset_path_count())
        for n in range(len(n_best_arr)):
            for m in range(l_size//n_best-1):
                if rule_based_mutations:
                    if random.random() < 0.32:
                        layouts_arr.append(mutate_uniformity(n_best_arr[n], alpha=alpha))
                        mutation_type = "LU" # layout uniformity
                    elif random.random() < 0.64:
                        layouts_arr.append(mutate_missing_items(n_best_arr[n], list(flatten(res_dict[random.randint(0, n_best-1)])), alpha=alpha))
                        mutation_type = "MI" # missing items
                    elif random.random() < 0.96:
                        layouts_arr.append(mutate_uniformity_for_rack(n_best_arr[n], alpha=alpha))
                        mutation_type = "RU" # rack uniformity
                    else:
                        layouts_arr.append(mutate_layout(n_best_arr[n], alpha=alpha))
                        mutation_type = "RI" # random item
                else:
                    layouts_arr.append(mutate_layout(n_best_arr[n], alpha=alpha))
    print(f'Iteration {n_iter} \'{mutation_type}\'', end=' ')
    res, hi, res_dict = select_n_best_layouts(layouts_arr, check_arr, n_best=n_best, start_pos=start_pos, debug=debug, reward_type=reward_type, use_item_count=use_item_count, weights=weights)
    history.append(hi)
    return res, history

In [34]:
score_best_layouts, score_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=100, n_best=2, alpha=0.05, debug=True, reward_type='score', use_item_count=True, weights=(700, 150, 150))

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 0 '' Scores: [735.75, 735.75]
Iteration 1 '' Scores: [717.2200704225353, 727.8221830985916]
Iteration 2 '' Scores: [710.5457746478874, 717.2200704225353]
Iteration 3 '' Scores: [691.6214788732394, 703.4753521126761]
Iteration 4 '' Scores: [680.3573943661972, 684.6848591549297]
Iteration 5 '' Scores: [671.6038732394366, 674.1778169014084]
Iteration 6 '' Scores: [655.8521126760563, 657.2711267605633]
Iteration 7 '' Scores: [639.5369718309859, 641.9154929577466]
Iteration 8 '' Scores: [625.4031690140845, 631.9718309859155]
Iteration 9 '' Scores: [605.2570422535211, 612.225352112676]
Iteration 10 '' Scores: [592.0492957746479, 599.6760563380282]
Iteration 11 '' Scores: [586.4330985915492, 590.132042253521]
Iteration 12 '' Scores: [581.3468309859155, 585.4419014084507]
Iteration 13 '' Scores: [573.1232394366197, 580.1901408450705]
Iteration 14 '' Scores: [567.8714788732394, 567.9383802816901]
Iteration 15 '' Scores: [548.3221830985915, 561.5633802816901]
Iteration 16 '' Scores: [5

In [35]:
curr_dir = Path(os.getcwd()).parent

In [101]:
score_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [37]:
print(score_best_layouts[1].get_layout_json())

{"hideSaveLoadButtons":true,"rackLevels":4,"items":["baking ingredients","soy lactosefree","butter","fresh vegetables","yogurt","canned meals beans","poultry counter","ice cream ice","fresh fruits","milk","packaged cheese","bread","tea","bakery desserts","frozen breakfast","cereal","eggs","buns rolls","cream","water seltzer sparkling water","pickled goods olives","packaged poultry","other creams cheeses","honeys syrups nectars","coffee","refrigerated","energy granola bars","soft drinks","latino foods","plates bowls cups flatware","paper goods","oral hygiene","diapers wipes","food storage","nuts seeds dried fruit","soap","packaged vegetables fruits","hot dogs bacon sausage","lunch meat","chips pretzels","meat counter","fresh dips tapenades","prepared soups salads","condiments","juice nectars","canned fruit applesauce","preserved dips spreads","packaged produce","canned jarred vegetables","fresh pasta","pasta sauce","frozen produce","frozen appetizers sides","soup broth bouillon","dry pa

In [38]:
def save_layouts(layouts_to_save, metric):
    for i in range(len(layouts_to_save)):
        filename = f'./../data/layouts/genetic/{metric}/layout_{i}.json'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'w') as f:
            f.write(layouts_to_save[i].get_layout_json())

In [39]:
save_layouts(score_best_layouts, 'score')

In [99]:
def plot_score_history(history):
    fig = px.line(x=range(len(history)), y=[x[0] for x in history], title='Score', 
                  labels={'value': 'Score', 'x': 'Iteration'}, height=700, width=700)
    fig.show()

In [100]:
plot_score_history(score_history)

## max metric

In [42]:
max_best_layouts, max_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=100, n_best=2, alpha=0.05, debug=True, reward_type='max', use_item_count=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 0 '' ['P:18, I:399, U:0,142', 'P:18, I:399, U:0,142']
Iteration 1 '' ['P:549, I:385, U:137,182', 'P:475, I:382, U:117,177']
Iteration 2 '' ['P:1069, I:374, U:207,204', 'P:1035, I:370, U:196,202']
Iteration 3 '' ['P:2057, I:350, U:295,237', 'P:1879, I:358, U:279,229']
Iteration 4 '' ['P:2774, I:334, U:350,257', 'P:2774, I:335, U:368,265']
Iteration 5 '' ['P:4020, I:310, U:413,283', 'P:3795, I:317, U:432,287']
Iteration 6 '' ['P:4670, I:298, U:460,300', 'P:4617, I:303, U:479,307']
Iteration 7 '' ['P:5894, I:283, U:520,322', 'P:5258, I:288, U:492,313']
Iteration 8 '' ['P:6455, I:270, U:568,339', 'P:6345, I:275, U:563,338']
Iteration 9 '' ['P:7433, I:256, U:596,359', 'P:7181, I:259, U:602,353']
Iteration 10 '' ['P:8279, I:243, U:646,377', 'P:7952, I:243, U:624,370']
Iteration 11 '' ['P:8573, I:235, U:656,384', 'P:8573, I:235, U:657,385']
Iteration 12 '' ['P:9237, I:225, U:688,394', 'P:8903, I:230, U:690,399']
Iteration 13 '' ['P:9562, I:221, U:734,410', 'P:9488, I:222, U:706,405'

In [43]:
save_layouts(max_best_layouts, 'max')

In [93]:
def plot_history(history, mode='max'):
    path = [x[0]['path'] for x in history]
    if mode == 'min':
        path = [x[0]['path'] - x[0]['invalid']*1000 for x in history]
    invalid = [x[0]['invalid'] for x in history]
    print(len(path), len(invalid))
    fig = px.line(x=range(len(history)), y=[path, invalid], title='Path count', 
                  labels={'value': 'Count', 'x': 'Iteration'}, height=700, width=700)
    new_names = {'wide_variable_0': 'Path', 'wide_variable_1': 'Invalid'}
    fig.for_each_trace(lambda t: t.update(name=new_names[t.name]))
    fig.show()

In [94]:
plot_history(max_history)

101 101


In [96]:
max_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


## min metric

In [47]:
min_best_layouts, min_history = genetic_algorithm(layouts_array, check_list[:400], n_iter=100, n_best=2, alpha=0.05, debug=True, reward_type='min', use_item_count=True)

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 0 '' ['P:399014, I:399, U:0,142', 'P:399014, I:399, U:0,142']
Iteration 1 '' ['P:385614, I:385, U:106,173', 'P:387401, I:387, U:94,169']
Iteration 2 '' ['P:364572, I:363, U:205,203', 'P:370174, I:369, U:210,204']
Iteration 3 '' ['P:344518, I:342, U:289,232', 'P:347401, I:345, U:289,232']
Iteration 4 '' ['P:328405, I:325, U:356,255', 'P:329341, I:326, U:353,253']
Iteration 5 '' ['P:312222, I:308, U:438,281', 'P:315185, I:311, U:416,277']
Iteration 6 '' ['P:302755, I:298, U:491,305', 'P:303510, I:299, U:472,296']
Iteration 7 '' ['P:283887, I:278, U:565,335', 'P:292313, I:287, U:536,325']
Iteration 8 '' ['P:269707, I:263, U:596,350', 'P:270567, I:264, U:600,348']
Iteration 9 '' ['P:257434, I:250, U:640,366', 'P:259339, I:252, U:643,372']
Iteration 10 '' ['P:241532, I:233, U:677,393', 'P:248161, I:240, U:698,388']
Iteration 11 '' ['P:230659, I:221, U:737,409', 'P:233365, I:224, U:694,403']
Iteration 12 '' ['P:225816, I:216, U:761,422', 'P:226781, I:217, U:753,420']
Iteration 13 '

In [48]:
save_layouts(min_best_layouts, 'min')

In [97]:
plot_history(min_history, 'min')

101 101


In [98]:
min_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


# Simulated Annealing

In [51]:
def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

check_config = {
    1: 30,
    2: 50,
    3: 60,
    4: 75,
    5: 75,
    6: 60,
    7: 50,
}

tuned_checks = get_checks_of_specific_length_range(check_list, check_config)


  0%|          | 0/7 [00:00<?, ?it/s]

In [52]:
def get_temperature(tested_layout, test_check_list, return_layout=False):
    _layout = tested_layout.copy()
    res, proc_layout = thread_func(_layout, test_check_list, use_item_count=True)
    if return_layout:
        return 1000 - calculate_score(res, proc_layout, test_check_list, (600, 200, 200)), proc_layout
    return 1000 - calculate_score(res, proc_layout, test_check_list, (600, 200, 200)), res

In [53]:
def simulated_annealing(initial_layout, initial_temperature, cooling_rate, checks, max_iter = 10000, prob=0.5, step=1, rule_based_mutations=False, alpha=0.05):
    current_layout = initial_layout
    temperature = initial_temperature
    history = {}
    res = {}
    _best_layout = current_layout
    best_temperature = 0
    for i in trange(max_iter):
        if temperature > 1:
            #new_layout = mutate_layout(current_layout, alpha=0.05)
            if rule_based_mutations:
                if random.random() < 0.32:
                    new_layout = mutate_uniformity(current_layout, alpha=alpha)
                elif random.random() < 0.64:
                    new_layout = mutate_missing_items(current_layout, list(flatten(res['missing_items'])), alpha=alpha)
                elif random.random() < 0.96:
                    new_layout = mutate_uniformity_for_rack(current_layout, alpha=alpha)
                else:
                    new_layout = mutate_layout(current_layout, alpha=alpha)
            else:
                new_layout = mutate_layout(current_layout, alpha=alpha)
            new_temperature, res = get_temperature(new_layout.copy(), checks)
            if new_temperature > temperature:
                history[i] = [new_temperature, 'higher']
                temperature = new_temperature
                current_layout = new_layout
                print(f'{i}: New layout accepted with temperature {new_temperature}')
            else:
                if random.random() < np.exp((new_temperature - temperature) / temperature - prob)**step:
                    history[i] = [new_temperature, 'lower']
                    current_layout = new_layout
                    print(f'{i}: New layout accepted with temperature {new_temperature}. Reason: random.')
            temperature *= (1 - cooling_rate)
        else:
            break
        if temperature > best_temperature:
            _best_layout = current_layout
            best_temperature = temperature
            print(f'\n*** New best layout found with temperature {best_temperature} ***\n')
    return current_layout, _best_layout, history

In [54]:
initial_layout = random_layout()

In [55]:
curr, best_layout, iter_history = simulated_annealing(initial_layout, 3, 0.01, tuned_checks, max_iter=4000, prob=0.7)

  0%|          | 0/4000 [00:00<?, ?it/s]

0: New layout accepted with temperature 302.3169014084507

*** New best layout found with temperature 299.2937323943662 ***
1: New layout accepted with temperature 301.87323943661977
2: New layout accepted with temperature 301.87323943661977
4: New layout accepted with temperature 315.37323943661977

*** New best layout found with temperature 312.21950704225355 ***
5: New layout accepted with temperature 324.7253521126761

*** New best layout found with temperature 321.4780985915493 ***
6: New layout accepted with temperature 334.0774647887324

*** New best layout found with temperature 330.7366901408451 ***
7: New layout accepted with temperature 338.2253521126761

*** New best layout found with temperature 334.84309859154934 ***
9: New layout accepted with temperature 338.2253521126761
10: New layout accepted with temperature 341.2253521126761

*** New best layout found with temperature 337.8130985915493 ***
11: New layout accepted with temperature 340.1690140845071
12: New layout ac

In [56]:
best_layout.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [57]:
score, layout = get_temperature(best_layout, tuned_checks, return_layout=True)

In [58]:
score

432.90140845070425

In [59]:
layout.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [60]:
def plot_temperature_history(history):
    # plot a line with red dots for higher, blue for lower
    fig = px.line(x=history.keys(), y=[x[0] for x in history.values()], title='Temperature', 
                  labels={'value': 'Temperature', 'x': 'Iteration'}, height=700)
    fig.add_scatter(x=[x for x in history.keys() if history[x][1] == 'higher'], y=[history[x][0] for x in history.keys() if history[x][1] == 'higher'], mode='markers', marker=dict(color='red'))
    fig.add_scatter(x=[x for x in history.keys() if history[x][1] == 'lower'], y=[history[x][0] for x in history.keys() if history[x][1] == 'lower'], mode='markers', marker=dict(color='blue'))
    # plot highest temperature as green dot
    sorted_dict = dict(sorted(history.items(), key=lambda item: item[1][0], reverse=True))
    fig.add_scatter(x=[list(sorted_dict.keys())[0]], y=[list(sorted_dict.values())[0][0]], mode='markers', marker=dict(color='green'))
    fig.show()

In [61]:
plot_temperature_history(iter_history)

In [70]:
curr_high_prob, best_layout_high_prob, iter_history = simulated_annealing(initial_layout, 3, 0.005, tuned_checks, max_iter=4000, prob=0.2, alpha=0.05, rule_based_mutations=True)

  0%|          | 0/4000 [00:00<?, ?it/s]

0: New layout accepted with temperature 314.4718309859154

*** New best layout found with temperature 312.8994718309858 ***
1: New layout accepted with temperature 326.2746478873239

*** New best layout found with temperature 324.64327464788727 ***
2: New layout accepted with temperature 322.99295774647885. Reason: random.
3: New layout accepted with temperature 325.4718309859154
4: New layout accepted with temperature 330.2112676056338

*** New best layout found with temperature 328.5602112676056 ***
5: New layout accepted with temperature 351.8943661971831

*** New best layout found with temperature 350.13489436619716 ***
6: New layout accepted with temperature 350.92957746478874
7: New layout accepted with temperature 343.4366197183099. Reason: random.
8: New layout accepted with temperature 335.12676056338023. Reason: random.
10: New layout accepted with temperature 333.87323943661966. Reason: random.
11: New layout accepted with temperature 335.09859154929575. Reason: random.
12: 

In [71]:
best_layout_high_prob.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [72]:
plot_temperature_history(iter_history)

In [66]:
score, layout = get_temperature(best_layout_high_prob, check_list[:200], return_layout=True)

NameError: name 'best_layout_high_prob' is not defined

In [67]:
score

432.90140845070425

In [68]:
layout.display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


# Combinations

In [73]:
# display diagram with check length
def plot_check_length(check_list):
    fig = px.histogram(x=[len(x[1]) for x in check_list], title='Check length', labels={'value': 'Length', 'x': 'Check'}, height=700)
    fig.show()

plot_check_length(check_list)

In [74]:
def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

In [75]:
check_config = {
    1: 30,
    2: 50,
    3: 60,
    4: 75,
    5: 75,
    6: 60,
    7: 50,
}

In [76]:
tuned_checks = get_checks_of_specific_length_range(check_list, check_config)

layouts_array = []
for i in range(50):
    layouts_array.append(layout_with_one_item(4))

  0%|          | 0/7 [00:00<?, ?it/s]

In [77]:
def alpha_change_func(i, alpha, n_iter):
    new_alpha = alpha
    last_num = i // 10
    if last_num == 0:
        new_alpha = 0.8
    elif last_num == 1:
        new_alpha = 0.7
    elif last_num == 2:
        new_alpha = 0.6
    elif last_num == 3:
        new_alpha = 0.5
    elif last_num == 4:
        new_alpha = 0.4
    elif last_num == 5:
        new_alpha = 0.3
    elif last_num == 6:
        new_alpha = 0.2
    elif last_num == 7:
        new_alpha = 0.1
    elif last_num == 8:
        new_alpha = 0.05
    elif last_num == 9:
        new_alpha = 0.01
    return new_alpha

In [78]:
def alpha_change_func2(i, alpha, n_iter):
    new_alpha = alpha
    if i % 10 == 0:
        new_alpha = alpha - 0.1
    if new_alpha < 0.02:
        new_alpha = 0.02
    return new_alpha

In [79]:
step1_best_layouts, step1_history = genetic_algorithm(
    layouts_array, 
    tuned_checks, 
    n_iter=50, 
    n_best=2, 
    alpha=0.05, 
    debug=True, 
    reward_type='max', 
    use_item_count=True, 
    rule_based_mutations=True)

  0%|          | 0/50 [00:00<?, ?it/s]

Iteration 0 '' ['P:0, I:400, U:0,142', 'P:0, I:400, U:0,142']
Iteration 1 'LU' ['P:803, I:378, U:135,183', 'P:694, I:380, U:104,174']
Iteration 2 'LU' ['P:3733, I:317, U:345,247', 'P:3141, I:329, U:333,248']
Iteration 3 'MI' ['P:5247, I:287, U:453,286', 'P:4928, I:296, U:438,288']
Iteration 4 'RU' ['P:5961, I:271, U:515,313', 'P:5806, I:278, U:508,305']
Iteration 5 'MI' ['P:6767, I:257, U:561,334', 'P:6708, I:260, U:573,336']
Iteration 6 'LU' ['P:8191, I:233, U:619,359', 'P:7931, I:241, U:616,356']
Iteration 7 'LU' ['P:9002, I:221, U:588,368', 'P:8844, I:223, U:664,380']
Iteration 8 'RU' ['P:9461, I:211, U:643,389', 'P:9353, I:214, U:647,389']
Iteration 9 'MI' ['P:9796, I:208, U:628,387', 'P:9739, I:207, U:671,400']
Iteration 10 'RU' ['P:10529, I:194, U:665,402', 'P:10438, I:200, U:664,401']
Iteration 11 'MI' ['P:11503, I:181, U:702,418', 'P:10862, I:189, U:692,412']
Iteration 12 'LU' ['P:11636, I:178, U:731,430', 'P:11521, I:180, U:733,429']
Iteration 13 'RU' ['P:12413, I:167, U:754,4

In [80]:
plot_history(step1_history)

51 51


In [81]:
save_layouts(step1_best_layouts, 'step1-max')

In [82]:
valid_layouts_mutated = []
for best in step1_best_layouts:
    for i in range(50):
        valid_layouts_mutated.append(mutate_layout(best, alpha=0.01))

In [83]:
step2_best_layouts, step2_history = genetic_algorithm(valid_layouts_mutated, tuned_checks, n_iter=90, n_best=2, alpha=0.01, debug=True, reward_type='score', use_item_count=True, weights=(300, 200, 500), rule_based_mutations=True)

  0%|          | 0/90 [00:00<?, ?it/s]

Iteration 0 '' Scores: [506.52112676056345, 512.5070422535211]
Iteration 1 'LU' Scores: [505.11267605633805, 505.11267605633805]
Iteration 2 'MI' Scores: [501.0633802816902, 501.7676056338028]
Iteration 3 'RU' Scores: [497.5035211267606, 499.12676056338023]
Iteration 4 'MI' Scores: [493.806338028169, 494.86267605633805]
Iteration 5 'MI' Scores: [489.9330985915493, 490.9894366197183]
Iteration 6 'LU' Scores: [485.53169014084506, 486.23591549295776]
Iteration 7 'MI' Scores: [483.59507042253523, 484.12323943661977]
Iteration 8 'MI' Scores: [479.7218309859155, 479.8978873239437]
Iteration 9 'RU' Scores: [476.37676056338023, 477.08098591549293]
Iteration 10 'MI' Scores: [473.55985915492954, 473.5598591549296]
Iteration 11 'RU' Scores: [468.8978873239437, 470.39084507042253]
Iteration 12 'MI' Scores: [465.3767605633803, 466.25704225352115]
Iteration 13 'MI' Scores: [462.8661971830986, 463.4401408450704]
Iteration 14 'MI' Scores: [459.87323943661966, 459.919014084507]
Iteration 15 'LU' Scores

In [84]:
curr_dir = Path(os.getcwd()).parent
step2_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


# merging layout items ito categories

idea - create a dictionary with categories as keys and items as values
based on this dictionary mark layout sells with categories (more items from category - category is marked)
if there are more items from different categories - do not mark

In [85]:
categories = dict() # item - category
item_count = df['product_name'].unique().shape[0]

In [86]:
products = df[['product_name', 'department']].drop_duplicates()

In [87]:
for index, row in products.iterrows():
    categories[row['product_name']] = row['department']

In [88]:
sorted(categories.items(), key=lambda x: x[1])

[('beers coolers', 'alcohol'),
 ('spirits', 'alcohol'),
 ('specialty wines champagnes', 'alcohol'),
 ('white wines', 'alcohol'),
 ('red wines', 'alcohol'),
 ('diapers wipes', 'babies'),
 ('baby food formula', 'babies'),
 ('baby accessories', 'babies'),
 ('baby bath body care', 'babies'),
 ('bread', 'bakery'),
 ('bakery desserts', 'bakery'),
 ('buns rolls', 'bakery'),
 ('tortillas flat bread', 'bakery'),
 ('breakfast bakery', 'bakery'),
 ('tea', 'beverages'),
 ('water seltzer sparkling water', 'beverages'),
 ('coffee', 'beverages'),
 ('refrigerated', 'beverages'),
 ('soft drinks', 'beverages'),
 ('juice nectars', 'beverages'),
 ('energy sports drinks', 'beverages'),
 ('cocoa drink mixes', 'beverages'),
 ('cereal', 'breakfast'),
 ('hot cereal pancake mixes', 'breakfast'),
 ('granola', 'breakfast'),
 ('breakfast bars pastries', 'breakfast'),
 ('bulk grains rice dried goods', 'bulk'),
 ('bulk dried fruits vegetables', 'bulk'),
 ('canned meals beans', 'canned goods'),
 ('canned fruit apples

In [89]:
def get_category_for_item(item):
    return categories[item]

def set_category_for_items(layout):
    def select_category(items):
        counts = dict()
        for item in items:
            if item in categories.keys():
                if categories[item] in counts.keys():
                    counts[item] += 1
                else:
                    counts[item] = 1
        if len(counts) == 4:
            return ""
        else:
            sorted(counts.items(), key=lambda x: x[1], reverse=True)
            return categories[list(counts.keys())[0]]
    
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                layout[row][col].category = select_category([x[0] for x in layout[row][col].items])
                if layout[row][col].category:
                    print(f"Category: {layout[row][col].category} for {[x[0] for x in layout[row][col].items]}")
    return layout

In [90]:
step2_best_layouts[0] = set_category_for_items(step2_best_layouts[0])

Category: canned goods for ['canned jarred vegetables', 'canned jarred vegetables', 'canned jarred vegetables', 'canned jarred vegetables']
Category: canned goods for ['soup broth bouillon', 'soup broth bouillon', 'spirits', 'spirits']
Category: household for ['dish detergents', 'dish detergents', 'dish detergents', 'dish detergents']
Category: beverages for ['water seltzer sparkling water', 'water seltzer sparkling water', 'milk', 'water seltzer sparkling water']
Category: meat seafood for ['packaged seafood', 'packaged seafood', 'milk', 'milk']
Category: pantry for ['baking ingredients', 'baking ingredients', 'baking ingredients', 'baking ingredients']
Category: pantry for ['baking ingredients', 'baking ingredients', 'baking ingredients', 'baking ingredients']
Category: bakery for ['bread', 'ice cream toppings', 'bread', 'ice cream toppings']
Category: dairy eggs for ['butter', 'baking ingredients', 'butter', 'baking ingredients']
Category: pantry for ['baking ingredients', 'butter',

In [91]:
step2_best_layouts[0].display_in_window(home_dir=str(curr_dir), debug=True)

[pywebview] Using GTK


In [92]:
step2_best_layouts[0].get_layout_json()

'{"hideSaveLoadButtons":true,"rackLevels":4,"items":["baking ingredients","soy lactosefree","butter","fresh vegetables","yogurt","canned meals beans","poultry counter","ice cream ice","fresh fruits","milk","packaged cheese","bread","tea","bakery desserts","frozen breakfast","cereal","eggs","buns rolls","cream","water seltzer sparkling water","pickled goods olives","packaged poultry","other creams cheeses","honeys syrups nectars","coffee","refrigerated","energy granola bars","soft drinks","latino foods","plates bowls cups flatware","paper goods","oral hygiene","diapers wipes","food storage","nuts seeds dried fruit","soap","packaged vegetables fruits","hot dogs bacon sausage","lunch meat","chips pretzels","meat counter","fresh dips tapenades","prepared soups salads","condiments","juice nectars","canned fruit applesauce","preserved dips spreads","packaged produce","canned jarred vegetables","fresh pasta","pasta sauce","frozen produce","frozen appetizers sides","soup broth bouillon","dry p